In [1]:
from google.colab import drive
drive.mount('/content/drive')

import nbformat

with open('/content/drive/My Drive/Colab Notebooks/DIP_proj/pretrained_second_layer.ipynb') as f:
    notebook = nbformat.read(f, as_version=4)

# Execute each cell
for cell in notebook.cells:
    if cell.cell_type == 'code':
        exec(cell.source)


Mounted at /content/drive


In [22]:
import numpy as np
def get_inception_data():
    data = np.load('/content/drive/My Drive/Colab Notebooks/DIP_proj/inception_embeddings_x_train.npz')
    x_train_embeddings = data['embeddings']
    y_train = data['labels']

    data = np.load('/content/drive/My Drive/Colab Notebooks/DIP_proj/inception_embeddings_x_test.npz')
    x_test_embeddings = data['embeddings']
    y_test = data['labels']
    return x_train_embeddings, y_train, x_test_embeddings, y_test

def get_cifar_data():
    data = np.load('/content/drive/My Drive/Colab Notebooks/DIP_proj/vgg16_embeddings_x_train.npz')
    x_train_embeddings = data['embeddings']
    y_train = data['labels']

    data = np.load('/content/drive/My Drive/Colab Notebooks/DIP_proj/vgg16_embeddings_x_test.npz')
    x_test_embeddings = data['embeddings']
    y_test = data['labels']
    return x_train_embeddings, y_train, x_test_embeddings, y_test

def get_resnet_data():
    data = np.load('/content/drive/My Drive/Colab Notebooks/DIP_proj/resnet50_embeddings_x_train.npz')
    x_train_embeddings = data['embeddings']
    y_train = data['labels']

    data = np.load('/content/drive/My Drive/Colab Notebooks/DIP_proj/resnet50_embeddings_x_test.npz')
    x_test_embeddings = data['embeddings']
    y_test = data['labels']
    return x_train_embeddings, y_train, x_test_embeddings, y_test


In [3]:
import numpy as np

def create_federated_data(x_train, y_train, num_clients):
    client_data = []
    num_samples_per_client = len(x_train) // num_clients

    for i in range(num_clients):
        start_index = i * num_samples_per_client
        end_index = start_index + num_samples_per_client
        client_data.append((x_train[start_index:end_index], y_train[start_index:end_index]))

    return client_data

# num_clients = 5
# federated_data = create_federated_data(x_train_embeddings, y_train, num_clients)

In [46]:

cifar_labels = [
    'airplane', 'automobile', 'bird', 'cat',
    'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
]

# Define new categories
category_mapping = {
    'Roadways (land)': ['automobile', 'truck'],
    'Vehicles (air and sea)': ['airplane', 'ship'],
    'Birds': ['bird'],
    'Domestic Animals': ['cat', 'dog'],
    'Wild/Other Animals': ['deer', 'frog', 'horse']
}

def create_federated_data_by_class(x_train, y_train):
    label_to_category = {}
    for category, labels in category_mapping.items():
      for label in labels:
        label_index = cifar_labels.index(label)
        label_to_category[label_index] = category
    return label_to_category

# label_to_category = create_federated_data_by_class(x_train_embeddings, y_train)

def split_data_by_category(x_data, y_data, model_type, label_to_category):
    # Create a dictionary to hold the new categories
    categorized_data = {category: [] for category in category_mapping.keys()}
    categorized_labels = {category: [] for category in category_mapping.keys()}

    for img, label in zip(x_data, y_data):
        if model_type == 'inception':
            label_index = label[0]
        else:
            label_index = label[0]
        category = label_to_category.get(label_index)

        if category:
            categorized_data[category].append(img)
            categorized_labels[category].append(label_index)

    # Convert lists to numpy arrays for consistency
    for category in categorized_data.keys():
        categorized_data[category] = np.array(categorized_data[category])
        categorized_labels[category] = np.array(categorized_labels[category])
    cat_labels=[]
    for each in categorized_labels:
        cat_labels.append(list(categorized_labels[each]))
    return categorized_data, cat_labels

In [5]:
# split_x_train, split_y_train = split_data_by_category(x_train_embeddings, y_train, model_type, label_to_category)


In [6]:
# split_x_train.keys()


In [7]:
# federated_data[0][0].shape

In [8]:
def encode_weights(weights):
    return [w + 1 for w in weights]

def decode_weights(weights):
    return [w - 1 for w in weights]

In [9]:
def aggregate_models(models):
    # weights = [model.get_weights() for model in models]
    decrypted_weights = [decode_weights(weights) for weights in models]
    avg_weights = [np.mean(np.array(w), axis=0) for w in zip(*decrypted_weights)]
    return avg_weights

In [10]:
def train_on_client(model, client_data, epochs=25):
    x_train, y_train = client_data
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, verbose=0)


In [11]:
def train_model(fedarated_data, test_data, model_type):
# Initialize the model
    if model_type == 'resnet':
        global_model = create_model_resnet()
    if model_type == 'cifar':
        global_model = create_model_cifar()
    if model_type == 'inception':
        global_model = create_model_inception()
    # global_model = create_model()
    global_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
    # Federated training process
    num_rounds = 50
    for round_num in range(num_rounds):
        print(f'Round {round_num + 1}')
        encrypted_updates = []

        for client_data in federated_data:
            if model_type == 'resnet':
                client_model = create_model_resnet()
            if model_type == 'cifar':
                client_model = create_model_cifar()
            if model_type == 'inception':
                client_model = create_model_inception()

            client_model.set_weights(global_model.get_weights())
            train_on_client(client_model, client_data, epochs=1)
            # local_models.append(client_model)

            # Encrypt the weights
            encrypted_weights = encode_weights(client_model.get_weights())
            encrypted_updates.append(encrypted_weights)

        # Aggregate models
        global_weights = aggregate_models(encrypted_updates)
        global_model.set_weights(global_weights)

    # Evaluate the global model
    global_model.evaluate(test_data[0], test_data[1])
    return global_model


In [12]:
x_train_embeddings, y_train, x_test_embeddings, y_test = get_inception_data()

In [13]:
num_clients = 5
federated_data = create_federated_data(x_train_embeddings, y_train, num_clients)

In [14]:
model_type = 'inception'
incep_model_rf = train_model(federated_data, [x_test_embeddings, y_test],model_type)

Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8393 - loss: 0.5266


In [15]:
pred= incep_model_rf.predict(x_test_embeddings)
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_test, np.argmax(pred, axis=1)))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
              precision    recall  f1-score   support

           0       0.93      0.82      0.88       187
           1       0.94      0.90      0.92       189
           2       0.84      0.78      0.81       201
           3       0.71      0.76      0.73       198
           4       0.60      0.85      0.71       181
           5       0.82      0.81      0.82       191
           6       0.94      0.77      0.85       213
           7       0.90      0.84      0.86       226
           8       0.89      0.93      0.91       203
           9       0.89      0.91      0.90       211

    accuracy                           0.84      2000
   macro avg       0.85      0.84      0.84      2000
weighted avg       0.85      0.84      0.84      2000



In [16]:
incep_model_rf.save("/content/drive/My Drive/Colab Notebooks/DIP_proj/inception_rf.h5")

In [17]:
label_to_category = create_federated_data_by_class(x_train_embeddings, y_train)

split_x_train, split_y_train = split_data_by_category(x_train_embeddings, y_train, model_type, label_to_category)


In [18]:
incep_model_cd = train_model([split_x_train, split_y_train], [x_test_embeddings, y_test],model_type)


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8439 - loss: 0.5024


In [19]:
pred= incep_model_cd.predict(x_test_embeddings)
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_test, np.argmax(pred, axis=1)))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       187
           1       0.95      0.88      0.92       189
           2       0.82      0.84      0.83       201
           3       0.70      0.75      0.73       198
           4       0.74      0.78      0.76       181
           5       0.88      0.75      0.81       191
           6       0.84      0.89      0.87       213
           7       0.89      0.85      0.87       226
           8       0.97      0.87      0.91       203
           9       0.87      0.93      0.90       211

    accuracy                           0.85      2000
   macro avg       0.85      0.84      0.84      2000
weighted avg       0.85      0.85      0.85      2000



In [20]:
incep_model_cd.save("/content/drive/My Drive/Colab Notebooks/DIP_proj/inception_cd.h5")

In [23]:
x_train_embeddings, y_train, x_test_embeddings, y_test = get_cifar_data()


In [24]:
num_clients = 5
federated_data = create_federated_data(x_train_embeddings, y_train, num_clients)

In [25]:
model_type = 'cifar'
cifar10_model_rf = train_model(federated_data, [x_test_embeddings, y_test],model_type)

Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8186 - loss: 1.2055


In [26]:
pred= cifar10_model_rf.predict(x_test_embeddings)
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_test, np.argmax(pred, axis=1)))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.75      0.91      0.82       187
           1       0.92      0.92      0.92       189
           2       0.80      0.76      0.78       201
           3       0.70      0.69      0.70       198
           4       0.72      0.77      0.74       181
           5       0.78      0.72      0.75       191
           6       0.87      0.83      0.85       213
           7       0.81      0.84      0.83       226
           8       0.95      0.85      0.90       203
           9       0.90      0.88      0.89       211

    accuracy                           0.82      2000
   macro avg       0.82      0.82      0.82      2000
weighted avg       0.82      0.82      0.82      2000



In [27]:
cifar10_model_rf.save("/content/drive/My Drive/Colab Notebooks/DIP_proj/cifar10_rf.h5")

In [47]:
label_to_category = create_federated_data_by_class(x_train_embeddings, y_train)

split_x_train, split_y_train = split_data_by_category(x_train_embeddings, y_train, model_type, label_to_category)


In [48]:
cifar10_model_cd = train_model([split_x_train, split_y_train], [x_test_embeddings, y_test],model_type)


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8780 - loss: 0.6317


In [49]:
pred= cifar10_model_cd.predict(x_test_embeddings)
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_test, np.argmax(pred, axis=1)))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
              precision    recall  f1-score   support

           0       0.93      0.86      0.89       187
           1       0.96      0.86      0.91       189
           2       0.88      0.83      0.85       201
           3       0.76      0.81      0.79       198
           4       0.76      0.90      0.83       181
           5       0.82      0.84      0.83       191
           6       0.94      0.89      0.92       213
           7       0.95      0.88      0.91       226
           8       0.93      0.92      0.93       203
           9       0.84      0.97      0.90       211

    accuracy                           0.88      2000
   macro avg       0.88      0.87      0.88      2000
weighted avg       0.88      0.88      0.88      2000



In [50]:
cifar10_model_cd.save("/content/drive/My Drive/Colab Notebooks/DIP_proj/cifar10_cd.h5")

In [34]:
x_train_embeddings, y_train, x_test_embeddings, y_test = get_resnet_data()


In [35]:
num_clients = 5
federated_data = create_federated_data(x_train_embeddings, y_train, num_clients)

In [36]:
model_type = 'resnet'
resnet50_model_rf = train_model(federated_data, [x_test_embeddings, y_test],model_type)

Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8855 - loss: 0.6835


In [37]:
y_test

array([[2],
       [1],
       [5],
       ...,
       [3],
       [2],
       [5]], dtype=uint8)

In [38]:
pred= resnet50_model_rf.predict(x_test_embeddings)
np.argmax(pred, axis=1)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([2, 1, 3, ..., 2, 6, 5])

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, np.argmax(pred, axis=1)))

              precision    recall  f1-score   support

           0       0.95      0.86      0.90       187
           1       0.96      0.93      0.94       189
           2       0.91      0.82      0.86       201
           3       0.72      0.83      0.77       198
           4       0.84      0.86      0.85       181
           5       0.83      0.84      0.83       191
           6       0.93      0.90      0.91       213
           7       0.93      0.91      0.92       226
           8       0.92      0.95      0.93       203
           9       0.89      0.94      0.92       211

    accuracy                           0.88      2000
   macro avg       0.89      0.88      0.88      2000
weighted avg       0.89      0.88      0.88      2000



In [40]:
resnet50_model_rf.save("/content/drive/My Drive/Colab Notebooks/DIP_proj/resnet50_rf.h5")

In [51]:
label_to_category = create_federated_data_by_class(x_train_embeddings, y_train)

split_x_train, split_y_train = split_data_by_category(x_train_embeddings, y_train, model_type, label_to_category)


In [52]:
resnet50_model_cd = train_model([split_x_train, split_y_train], [x_test_embeddings, y_test],model_type)


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Round 11
Round 12
Round 13
Round 14
Round 15
Round 16
Round 17
Round 18
Round 19
Round 20
Round 21
Round 22
Round 23
Round 24
Round 25
Round 26
Round 27
Round 28
Round 29
Round 30
Round 31
Round 32
Round 33
Round 34
Round 35
Round 36
Round 37
Round 38
Round 39
Round 40
Round 41
Round 42
Round 43
Round 44
Round 45
Round 46
Round 47
Round 48
Round 49
Round 50
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8686 - loss: 0.7477


In [53]:
pred= resnet50_model_cd.predict(x_test_embeddings)
np.argmax(pred, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_test, np.argmax(pred, axis=1)))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
              precision    recall  f1-score   support

           0       0.93      0.86      0.89       187
           1       0.91      0.95      0.93       189
           2       0.86      0.86      0.86       201
           3       0.78      0.79      0.79       198
           4       0.75      0.84      0.79       181
           5       0.78      0.88      0.83       191
           6       0.96      0.86      0.91       213
           7       0.95      0.89      0.92       226
           8       0.91      0.94      0.92       203
           9       0.95      0.88      0.91       211

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.88      0.88      0.88      2000



In [54]:
resnet50_model_cd.save("/content/drive/My Drive/Colab Notebooks/DIP_proj/resnet50_cd.h5")